## The new intake GUI - built with panel

It is often useful to explore catalog data using a Graphical User Interface. The previous iteration of the intake GUI was built in ipywidgets - which limited you to use from within the notebook. The new version is implemented in [panel](https://panel.pyviz.org) which means that in addition to rendering in the notebook - it also can easily be served and deployed as a tornado application. 


**NOTE:** This work depends on the latest released versions of panel and bokeh

In [ ]:
import intake
intake.output_notebook()

The intake GUI is composed of various sub-panels that can be opened and closed from the control pane on the left. Try clicking the + button to get started. With that pane open, navigate to a catalog of interest and click the "Add Catalog" button.

In [ ]:
intake.gui

The "Add Catalog" mechanism is also exposed through the API:

In [ ]:
intake.gui.add('./us_crime/us_crime.yaml')

Try loading remote catalogs available at: 
 - https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml
 - https://raw.githubusercontent.com/ContinuumIO/anaconda-package-data/master/catalog/anaconda_package_data.yaml
 - https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml

Once an entry has been selected from the "Entries" pane, you can access that item from `intake.gui`:

In [ ]:
intake.gui.item

This item is a regular intake catalog entry object that can be used in the regular ways. 

In [ ]:
intake.gui.item.to_dask().head()

You can also look at the GUI just for that entry:

In [ ]:
intake.gui.item.gui

OR you can get just one sub-panel from the gui and work with that directly:

In [ ]:
intake.gui.description

You can also initialize a gui straight from a catalog to jump directly into the part of the gui that deals with selecting and viewing entries. 

In [ ]:
cat = intake.open_catalog('./us_crime/us_crime.yaml')
cat.gui

Every `gui` exposes an `item` property that returns the selected catalog entry.

In [ ]:
us_crime = cat.gui.item

us_crime.plot.bivariate('Burglary rate', 'Property crime rate', legend=False, width=500, height=400) * \
us_crime.plot.scatter(  'Burglary rate', 'Property crime rate', color='black', size=15, legend=False) +\
us_crime.plot.table(['Burglary rate', 'Property crime rate'], width=350, height=350)

## Future Work

There are many plans for how this GUI could be expanded. In the near future, we anticipate adding a mechanism for inputting user parameters, adding some basic custom plotting, and allowing the editing and saving of catalogs. See [GH#225](https://github.com/intake/intake/issues/225) for a full discussion of the plan.